# 航迹模拟

- 1. 要素融合
    数据处理的前置操作，仅仅对于此项目有用。
- 2. 航迹生成
    通过给定的要素及给定参数生成航迹。
- 3. 格式转换
    将生成的航迹转换为指定格式，通常是 gpx 格式。



## 环境配置

工作空间等配置，以及要素拷贝，要素融合

In [1]:
import os
import arcpy

folder = r"E:\scripts\航迹模拟"
gdb_path = os.path.join(folder, 'data', '航迹模拟.gdb')

if not arcpy.Exists(gdb_path):
    arcpy.management.CreateFileGDB(os.path.dirname(gdb_path), os.path.basename(gdb_path))

arcpy.env.workspace = gdb_path
arcpy.env.overwriteOutput = True

# 1. 列出data目录下的非.gdb文件夹列表
data_dir = os.path.join(folder, 'data')
non_gdb_folders = [f for f in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, f)) and not f.endswith('.gdb')]
non_trajectory_folders = [f for f in non_gdb_folders if not f.endswith("_trajectory")]

# 2. 获取每个文件夹下的.shp文件路径
shp_files = []
for folder in non_trajectory_folders:
    folder_path = os.path.join(data_dir, folder)
    shp_files.extend([os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.shp')])

shp_files = shp_files[:1]
data_features = []
# 3. 修改字段名并拷贝到gdb中
for shp_file in shp_files:
    # 创建临时文件
    temp_shp = arcpy.CreateScratchName("temp", "", "FeatureClass", arcpy.env.scratchGDB)
    arcpy.CopyFeatures_management(shp_file, temp_shp)
    
    fields = arcpy.ListFields(temp_shp)
    field_names = [field.name.lower() for field in fields]
    
    if 'lin_chang' in field_names:
        arcpy.AlterField_management(temp_shp, 'lin_chang', '林场')
    if 'lin_ban' in field_names:
        arcpy.AlterField_management(temp_shp, 'lin_ban', '林班')
    
    # 拷贝修改后的文件到gdb
    output_feature = os.path.join(gdb_path, "C" +os.path.basename(os.path.dirname(shp_file)))
    data_features.append(output_feature)
    arcpy.CopyFeatures_management(temp_shp, output_feature)
    
    # 删除临时文件
    arcpy.Delete_management(temp_shp)

# 定义目标坐标系为4510
target_sr = arcpy.SpatialReference(4510)

# 对每个shp文件直接定义4510投影
for shp_file in data_features:
    print(f"正在为 {shp_file} 定义投影...")
    arcpy.DefineProjection_management(shp_file, target_sr)
    print(f"{shp_file} 的投影已定义为 4510")

print("所有shp文件的投影已统一为4510")


dissloved_data_features = data_features

正在为 E:\scripts\航迹模拟\data\航迹模拟.gdb\C2022年伊图里河林业有限公司温河分公司人工造林 定义投影...
E:\scripts\航迹模拟\data\航迹模拟.gdb\C2022年伊图里河林业有限公司温河分公司人工造林 的投影已定义为 4510
所有shp文件的投影已统一为4510


## 要素融合（消除）

使用 **消除** 工具融合要素（按边界）。使用消除工具前，需要 **选择** 要消除的要素。

> **注意** 原始数据可能存在问题，例如消除后的要素有空洞等，这里需要手动处理。

<del>

``` python
eliminated_features_path = os.path.join(gdb_path, 'eliminated_features')

temp_layer = "__temp_layer__"
arcpy.MakeFeatureLayer_management(copied_features_path, temp_layer)
arcpy.SelectLayerByAttribute_management(temp_layer, "NEW_SELECTION", "1=1")
arcpy.Eliminate_management(temp_layer, eliminated_features_path, selection = False)

# 检查消除后的要素的坐标系
desc = arcpy.Describe(eliminated_features_path)
if desc.spatialReference.name == "Unknown" or desc.spatialReference.factoryCode != 4510:
    print("警告：消除后的要素没有正确的坐标系")
    # 定义 4510 坐标系
    target_sr = arcpy.SpatialReference(4510)
    # 投影要素到 4510 坐标系
    projected_features_path = os.path.join(gdb_path, 'projected_features')
    arcpy.Project_management(eliminated_features_path, projected_features_path, target_sr)
    print(f"已将要素投影到4510坐标系")
    data_features_path = projected_features_path
else:
    print(f"消除后的要素坐标系：{desc.spatialReference.name} (WKID: {desc.spatialReference.factoryCode})")
    data_features_path = eliminated_features_path

with arcpy.da.SearchCursor(data_features_path, ["SHAPE@","*"]) as cursor:
    for row in cursor:
        polygon = row[0]
        if polygon.partCount != 1:
            print(row)
```

</del>

## 随机时间

假设现阶段每天有10个人出外业，工作时间从早9:30到下午2:00（防止工作时长过长），从这一时间段为每个要素随机航迹起始时间。


In [2]:
import random
import datetime
import pytz

# 设置日期和时间范围
start_date = datetime.date(2024, 9, 8)
end_date = datetime.date(2024, 9, 14)
start_time = datetime.time(9, 30)
end_time = datetime.time(14, 00)

# 创建北京时区对象
beijing_tz = pytz.timezone('Asia/Shanghai')

# 遍历dissloved_data_features中的每个要素类
for feature_class in dissloved_data_features:
    # 为要素类添加time字段（如果不存在）
    if not arcpy.ListFields(feature_class, "time"):
        arcpy.AddField_management(feature_class, "time", "TEXT", field_length=50)
    
    # 为该要素类生成随机日期
    random_date = start_date + datetime.timedelta(days=random.randint(0, (end_date - start_date).days))
    
    # 为每个要素生成随机时间并更新字段
    with arcpy.da.UpdateCursor(feature_class, ["time"]) as cursor:
        for row in cursor:
            # 生成随机时间
            random_time = datetime.datetime.combine(
                random_date,
                datetime.time(
                    hour=random.randint(start_time.hour, end_time.hour),
                    minute=random.randint(0, 59),
                    second=random.randint(0, 59)
                )
            )
            
            # 创建带时区的datetime对象
            beijing_time = beijing_tz.localize(random_time)
            # 转换为UTC时间
            utc_time = beijing_time.astimezone(pytz.UTC)
            
            # 格式化为UTC时间字符串
            utc_time_str = utc_time.strftime("%Y-%m-%dT%H:%M:%SZ")
            
            # 更新字段
            row[0] = utc_time_str
            cursor.updateRow(row)
    
    print(f"已为要素类 {feature_class} 添加随机UTC时间（日期：{random_date}）")

print("所有要素类处理完成")

已为要素类 E:\scripts\航迹模拟\data\航迹模拟.gdb\C2022年伊图里河林业有限公司温河分公司人工造林 添加随机UTC时间（日期：2024-09-11）
所有要素类处理完成


In [13]:
from trajectory_simulator import TrajectorySimulator
from trajectory_simulator.gpx_trajectory_observer import GPXTrajectoryObserver
from trajectory_simulator.arcgis_elevation_provider import ArcgisElevationProvider
from trajectory_simulator.config import Config
import os
import time
from shapely import Polygon,Point

# 设置日期和时间范围
start_date = datetime.date(2024, 9, 8)
end_date = datetime.date(2024, 9, 14)

config_path = os.path.join(os.getcwd(), "trajectory_simulator", "config.json")
print(config_path)

ytlh_dems_dir = r"C:\Users\ws\OneDrive\projs\basemaps\DEM高程\伊图里河\tifs"
wh_dems_dir = r"C:\Users\ws\OneDrive\projs\basemaps\DEM高程\温河\tifs"

ytlh_dems = [os.path.join(ytlh_dems_dir, f) for f in os.listdir(ytlh_dems_dir) if f.endswith('.tif')]
wh_dems = [os.path.join(wh_dems_dir, f) for f in os.listdir(wh_dems_dir) if f.endswith('.tif')]

dems = ytlh_dems + wh_dems
# elevationProvider = ArcgisElevationProvider(dems)
# 根据DEM获取高程数据
# simulator = TrajectorySimulator(config_path, elevation_provider = elevationProvider)
config = Config()
config.load(config_path)
simulator = TrajectorySimulator(config)

index_dict = {}
def naming_callback(row):
    key = f"{row[0]}_{int(row[1])}"
    if key not in index_dict:
        index_dict[key] = 1
    index = index_dict[key]
    index_dict[key] += 1
    return f"{key}_第{index}条航迹"

callback_fields = ['林场', '林班']
# 遍历dissolved_data_features中的每个要素类
for fc in dissloved_data_features:
    # 为每个要素类创建输出文件夹
    output_folder = os.path.join(folder, 'data', f"{os.path.basename(fc).split('.')[0]}_trajectory")
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    print(output_folder)
    # 生成航迹
    with arcpy.da.SearchCursor(fc, ["SHAPE@", "林场", "林班", "time"]) as cursor:
        for row in cursor:
            # 将ArcGIS的polygon转为shapely的polygon
            arcgis_polygon = row[0]
            shapely_polygon = Polygon([(point.X, point.Y) for point in arcgis_polygon.getPart(0) if point])
            
            time_of_row = row[-1]
            start_time = int(datetime.datetime.strptime(time_of_row, "%Y-%m-%dT%H:%M:%SZ").timestamp()) if time_of_row else int(time.time())  # 如果time字段为空，使用当前时间戳
            # 创建并添加GPX观察者
            gpx_config = {
                "creator": "ArcGIS Trajectory Simulator",
                "metadata_name": f"Simulated Inspection Trajectory",
                "track_name": f"Inspection Route",
                "metadata_description": f"A simulated trajectory for inspection tasks - Polygon",
                "metadata_author": "Your Name or Organization"
            }

            gpx_file_name = f"{row[1]}_{row[2]}_{row[-1]}.gpx"
            gpx_path = os.path.join(output_folder, gpx_file_name)
            gpx_observer = GPXTrajectoryObserver(gpx_path, gpx_config)
            simulator.add_observer(gpx_observer)
            x, y = shapely_polygon.exterior.coords[0]
            # 修正simulate调用
            simulator.simulate(start_time,Point(x,y), shapely_polygon)
            simulator.remove_observer(gpx_observer)
    
    print(f"模拟完成，{fc}的GPX文件保存在：{output_folder}")
    # 需要沿着终点继续走

print("所有要素类的航迹生成完成")

e:\scripts\航迹模拟\trajectory_simulator\config.json
配置已从文件 e:\scripts\航迹模拟\trajectory_simulator\config.json 加载
莫那根27林班\data\C2022年伊图里河林业有限公司温河分公司人工造林_trajectory


AttributeError: 'tuple' object has no attribute 'x'

## 航迹生成

简化真实场景，模拟航迹。

算法描述：
从给定多边形的顶点集合中随机选择一个点作为起点。

从起点开始，按顺序遍历多边形的顶点集合，直到返回起点。

对于每对相邻的顶点A和B：

1. 计算从A到B的方向向量。
2. 初始化当前点为A。
3. 循环生成航迹点，直到到达B或其延长线上：

- 生成随机速度（在给定范围内）。
- 生成随机方向偏移（在给定角度范围内）。
- 计算新的航迹点。
- 检查新点是否到达或超过B。
- 如果到达或超过B，将最后一个点设为B，否则添加新点到航迹。

### 关键点

- 起点问题：起点无需与 Polygon 的顶点重合。
- 速度问题：速度应为正值。
- 面积精度问题：拟合后的航迹所形成的 Polygon 的面积应达到原 Polygon 面积的 95%（可调节） 以上。
- GPX 文件生成：坐标转换，需要使用 WGS84 坐标系。
- 高程：是否需要获取高程数据？可以先生成航迹，再获取高程。
- 时间模拟：
    - 距离采样（定位过滤）：10米一采样
    - 时间采样：根据速度计算时间
- 算法优化：使用并发算法，将问题拆解为每段轨迹，使用多线程模拟。

### 真实数据-数据结构

都是根据位置和高程自动计算出来的

- 时间
- 海拔高度
- 航段长度
- 航段时间（行走用时）
- 航段方向（360°）
- 航段速度（km/h）

### BUG修复

- [√] 回头问题
- [√] 路径不闭合问题